In [5]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
chrome_options = Options()

In [6]:
previous_df=pd.read_csv(r"C:\Users\Dell\Desktop\file1.csv")

origin_IATA, destination_IATA, distance_km, flight_duration, airline, aircraft=[],[],[],[],[],[]
IATA_codes=previous_df['Origin_IATA'].unique().tolist()

In [ ]:
### because of errors received,I decided to save dataframe for each airport separately  
for airport in IATA_codes:
    filename = f'{airport}.csv'
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    wait = WebDriverWait(driver, 5)
    # handle the autocomplete of the search_box
    search_box= driver.find_element(By.XPATH,'/html/body/div/div[1]/div[1]/div[5]/input[1]')  
    search_box.send_keys(airport)
    autocomplete_options = WebDriverWait(driver, 10).until(
        EC.visibility_of_all_elements_located((By.XPATH, '//div[@id="quicksearch"]//ul[@class="autocomplete"]/li'))
    )
    # Click on the first autocomplete option
    if autocomplete_options:
        autocomplete_options[0].click()
    search_box.send_keys(Keys.ENTER)
    # click on 'search' button
    search_button=driver.find_element(By.XPATH,'/html/body/div/div[1]/div[1]/div[5]/input[3]') 
    search_button.click()
    # Wait for the element to be present
    airport_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div/div[1]/div[1]/div[2]/a'))
    )
    airport_button.click()
    # moving to the HTML source, in order to 'ease' on the driver
    table=driver.find_element(By.ID,'result')
    html = table.get_attribute('innerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    driver.quit()
    # now, extracte the relevant data on the airport
    rows = soup.find_all('tr')
    for row in rows:
        columns=row.find_all('td')
        if len(columns)<7:
            continue
        origin_IATA.append(columns[1].text)
        destination_IATA.append(columns[2].text)
        airline.append(columns[3].text)
        distance_km.append(columns[5].text)
        flight_duration.append(columns[6].text)
        aircraft.append(columns[7].text)
        
    df=pd.DataFrame({"origin_IATA": origin_IATA,
                 "destination_IATA":destination_IATA,
                 "airline":airline,
                 "distance (km)":distance_km,
                 "flight_duration":flight_duration,
                 "aircraft type":aircraft}) 
    # create dataframe for each airport, then aggregate them and merge 
    df.to_csv(filename, index=False)

In [7]:
folder_path = r'C:\Users\Dell\Desktop\dataframes'  
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)

merged_df = pd.concat(dataframes, ignore_index=True)
merged_df.to_csv('df2.csv', index=False)

In [8]:
df2=pd.read_csv('df2.csv')
df2

,origin_IATA,destination_IATA,airline,distance (km),flight_duration,aircraft type
0,AER,DME,S7 Airlines (S7),830,02:09,319 320 738\n
1,AER,DME,Ural Airlines (U6),830,02:09,320\n
2,AER,DME,Transaero Airlines (UN),830,02:09,734 735\n
3,AER,DME,NordStar Airlines (Y7),830,02:09,738\n
4,AER,DYU,Tajik Air (7J),1534,03:34,735\n
...,...,...,...,...,...,...
10299,ZRH,YUL,Swiss International Air Lines (LX),3723,07:56,333\n
10300,ZRH,YYZ,Air Canada (AC),4030,08:33,333\n
10301,ZRH,YYZ,Swiss International Air Lines (LX),4030,08:33,333\n
10302,ZRH,ZAG,Swiss International Air Lines (LX),375,01:15,DH4\n
